In [1]:
!nvidia-smi

Tue Oct 22 10:07:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.4     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:CA:00.0 Off |                    0 |
| N/A   32C    P0              95W / 400W |  16855MiB / 81920MiB |      1%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!git clone https://github.com/RefalMachine/llmtf_open
%cd llmtf_open
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl

fatal: destination path 'llmtf_open' already exists and is not an empty directory.
/workdir/diploma-llm/hw2/llmtf_open
--2024-10-22 10:07:48--  https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


200 OK
Length: 1291979 (1.2M) [text/plain]
Saving to: ‘train.jsonl.1’

train.jsonl.1       100%[===================>]   1.23M  --.-KB/s    in 0.1s    

2024-10-22 10:07:49 (8.78 MB/s) - ‘train.jsonl.1’ saved [1291979/1291979]



In [3]:
!ls

Dockerfile	      llmtf
README.md	      requirements.txt
RuOpinionNE_test1     run_evaluate_multinode_multigpu.py
RuOpinionNE_test2     run_evaluate_multinode_multigpu.sh
RuOpinionNE_test3     run_evaluate_singlenode_multigpu.sh
conversation_configs  todo.txt
eval_grammar.py       train.jsonl
evaluate_model.py     train.jsonl.1
examples


In [44]:
from llmtf.model import HFModel

model_name_or_path = 'Qwen/Qwen2.5-3B-Instruct'
model = HFModel(conversation_template_path='conversation_configs/qwen2.json', device_map='cuda:0', attn_implementation="sdpa")
model.from_pretrained(model_name_or_path)

model.generation_config.max_new_tokens = 200
model.generation_config.repetition_penalty = 1.0
model.generation_config.do_sample = False
model.generation_config.temperature = 0.4
model.generation_config

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO: 2024-10-22 11:13:26,093: llmtf.base.hfmodel: Set eos_token_id in generation_config to [151645]
INFO: 2024-10-22 11:13:26,094: llmtf.base.hfmodel: Model id: Qwen/Qwen2.5-3B-Instruct
INFO: 2024-10-22 11:13:26,095: llmtf.base.hfmodel: Leading space: False


GenerationConfig {
  "eos_token_id": [
    151645
  ],
  "max_length": 32768,
  "max_new_tokens": 200,
  "pad_token_id": 151643,
  "stop_strings": [
    "<|im_end|>"
  ],
  "temperature": 0.4,
  "top_k": 40,
  "top_p": 0.9,
  "trust_remote_code": false
}

In [103]:
from llmtf.tasks.ruopinionne import RuOpinionNE, pred2opinions_default
instruction = '''Нужно проанализировать текст и извлечь из него мнения. Они должны быть представлены в виде кортежа мнений, который состоит из 4 компонент:
1. Источник мнения (Source): автор (AUTHOR), именованная сущность (некоторая подстрока), если мнение присутствует, иначе NULL; 
2. Объект мнения (Target): именованная сущность в тексте (некоторая подстрока); 
3. Тональность (Polarity): позитивная/негативная (POS/NEG); 
4. Аргумент в пользу тональности (Expression): аргумент, на основании которого определена тональность; Аргумент это некоторая подстрока из текста, по которой определилась тональность.

Ответ нужно представить в формате json, элементы - кортежы мнений. 
Котреж мнений это словарь следующего вида: Source: ..., Target: ..., Polarity: ..., Expression: ...

### Текст
""" {text} """ 

Если сможешь предложить лучшее решение я награжу тебя золотой медалью.
'''

task = RuOpinionNE(instruction=instruction, pred2opinion=pred2opinions_default)

In [73]:
from llmtf.evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_dataset(
    task=task,
    model=model,
    output_dir='./RuOpinionNE_test1',
    max_len=4000,
    few_shot_count=5,
    generation_config=None, # will use model.generation_config by default
    batch_size=4,
    max_sample_per_dataset=200 # итоговую оценку нужно прислать для 200 сэмплов
)

INFO: 2024-10-22 11:35:48,563: llmtf.base.hfmodel: Updated generation_config.eos_token_id: [151645]
INFO: 2024-10-22 11:35:48,564: llmtf.base.hfmodel: Updated generation_config.stop_strings: ['<|im_end|>']
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 62.95it/s]
INFO: 2024-10-22 11:35:52,177: llmtf.base.ruopinionne: Loading Dataset: 3.61s
  4%|████████▋                                                                                                                                                                                                                 | 2/50 [00:04<01:49,  2.29s/it]

Fail to validate
{'Source': 'Дмитрий Медведев', 'Target': 'решение', 'Polarity': 'POS', 'Expression': 'по поручению'}
Как сообщает "Интерфакс" со ссылкой на пресс-службу МВД, это решение принял глава ведомства Рашид Нургалиев по поручению президента РФ Дмитрия Медведева.


  8%|█████████████████▍                                                                                                                                                                                                        | 4/50 [00:10<02:07,  2.77s/it]

Fail to validate
{'Source': 'Оппозиция', 'Target': 'администрацию Руссефф', 'Polarity': 'NEG', 'Expression': 'нарушив закон'}
Оппозиция утверждает, что администрация Руссефф нарушила закон, перераспределяя средства из бюджета с целью обеспечить себе общественную поддержку несмотря на слабеющую экономику в преддверии президентских выборов 2014 года.


 12%|██████████████████████████▏                                                                                                                                                                                               | 6/50 [00:17<02:16,  3.10s/it]

...

Fail to validate
{'Source': 'Олег Бетин', 'Target': 'развитие свиноводства', 'Polarity': 'POS', 'Expression': 'следует развивать'}
"Свиноводство следует развивать в закрытых комплексах, а не в личных подсобных хозяйствах", - считает губернатор Тамбовской области Олег Бетин.


 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎        | 48/50 [02:04<00:04,  2.42s/it]

Fail to validate
{'Source': 'Тамбовская жизнь', 'Target': 'обследование', 'Polarity': 'NEG', 'Expression': 'определили у Жиркова надрыв мышцы бедра левой ноги'}
Это стало известно после медицинского обследования в одной из израильских клиник, где врачи определили у Жиркова надрыв мышцы бедра левой ноги, сообщает "Тамбовская жизнь".


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:10<00:00,  2.60s/it]
INFO: 2024-10-22 11:38:02,396: llmtf.base.ruopinionne: Processing Dataset: 130.22s
INFO: 2024-10-22 11:38:02,396: llmtf.base.ruopinionne: Results for ruopinionne:
INFO: 2024-10-22 11:38:02,406: llmtf.base.ruopinionne: {'f1': 0.10206252499043901}


Fail to validate
{'Source': 'Житье-бытье', 'Target': 'слияние команды "Спартак" с "Академией футбола"', 'Polarity': 'POS', 'Expression': 'ранее речь шла о возможном'}
Ранее речь шла о возможном слиянии команды "Спартак" с "Академией футбола", пишет "Житье-бытье".
Fail to validate
{'Source': 'NULL', 'Target': 'Академия', 'Polarity': 'POS', 'Expression': 'наоборот'}
В зависимости от результатов первенства либо "Спартак" вливался в "Академию", либо наоборот.


In [75]:
!cat ./RuOpinionNE_test1/ruopinionne_total.jsonl

{
    "task_name": "ruopinionne",
    "results": {
        "f1": 0.10206252499043901
    },
    "leaderboard_result": 0.10206252499043901
}


In [9]:
!cat ./RuOpinionNE_test1/ruopinionne_params.jsonl

{
    "custom_generation_config": null,
    "model_params": {
        "model_name_or_path": "Qwen/Qwen2.5-3B-Instruct",
        "generation_config": {
            "eos_token_id": [
                151645
            ],
            "max_length": 32768,
            "max_new_tokens": 200,
            "pad_token_id": 151643,
            "stop_strings": [
                "<|im_end|>"
            ],
            "temperature": 0.4,
            "top_k": 40,
            "top_p": 0.9,
            "transformers_version": "4.45.2",
            "trust_remote_code": false
        },
        "conversation_template": {
            "system_prompt": "",
            "system_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
            "user_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
            "bot_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
            "bot_message_template_incomplete": "<|im_start|>{role}\n{content}",
            "user_role": "use

In [10]:
!cat ./RuOpinionNE_test1/ruopinionne.jsonl

{
    "metric": {
        "f1": {
            "gold": {
                "sent_id": 23,
                "text": "Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов.",
                "opinions": []
            },
            "preds": {
                "opinions": [],
                "sent_id": 23,
                "text": "Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов."
            }
        }
    },
    "predict": "***\n[]",
    "sample": {
        "sent_id": 23,
        "text": "Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов.",
        "opinions": []
    },
    "prompt": "<|im_start|>user\nНужно проанализировать текст и извлечь из него мнения. Они должны быть представлены в виде кортежа мнений, который состоит из 4 компонент:\n1. Источник мнения (Source): автор (AUTHOR), именованная сущность (некоторая подстр

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [100]:
from llmtf.tasks.ruopinionne import RuOpinionNE, pred2opinions_default
instruction = '''Нужно проанализировать текст и извлечь из него мнения. Они должны быть представлены в виде кортежа мнений, который состоит из 4 компонент:
1. Источник мнения (Source): автор (AUTHOR), именованная сущность (некоторая подстрока), если мнение присутствует, иначе NULL; 
2. Объект мнения (Target): именованная сущность в тексте (некоторая подстрока); 
3. Тональность (Polarity): позитивная/негативная (POS/NEG); 
4. Аргумент в пользу тональности (Expression): аргумент, на основании которого определена тональность; Аргумент это некоторая подстрока из текста, по которой определилась тональность.

Ответ нужно представить в формате json, элементы - кортежы мнений. 
Котреж мнений это словарь следующего вида: Source: ..., Target: ..., Polarity: ..., Expression: ...

### Текст
""" {text} """ 

### Пример вывода
Вывод должен выглядеть как-то так:
{{Source': 'Фонтанка.ру', 'Target': 'отношение общества', 'Polarity': 'POS', 'Expression': 'выяснить отношение'}}
Издание Фонтанка.ru провело опрос, с целью выяснить отношение общества к переименованию милиции.

Если сможешь предложить лучшее решение я награжу тебя золотой медалью.
'''

task = RuOpinionNE(instruction=instruction, pred2opinion=pred2opinions_default)

In [101]:
from llmtf.evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_dataset(
    task=task,
    model=model,
    output_dir='./RuOpinionNE_test2',
    max_len=4000,
    few_shot_count=5,
    generation_config=None, # will use model.generation_config by default
    batch_size=4,
    max_sample_per_dataset=200 # итоговую оценку нужно прислать для 200 сэмплов
)

INFO: 2024-10-22 12:08:54,490: llmtf.base.hfmodel: Updated generation_config.eos_token_id: [151645]
INFO: 2024-10-22 12:08:54,491: llmtf.base.hfmodel: Updated generation_config.stop_strings: ['<|im_end|>']
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 51.83it/s]
INFO: 2024-10-22 12:08:58,450: llmtf.base.ruopinionne: Loading Dataset: 3.96s
  2%|████▎                                                                                                                                                                                                                     | 1/50 [00:02<02:11,  2.67s/it]

Fail to validate
{'Source': 'NULL', 'Target': 'Берлускони', 'Polarity': 'POS', 'Expression': 'намерен баллотироваться'}
Читайте также: Берлускони намерен в 2013 г. снова баллотироваться на пост премьер-министра Италии


  4%|████████▋                                                                                                                                                                                                                 | 2/50 [00:05<02:00,  2.51s/it]

Fail to validate
{'Source': 'Фонтанка.ру', 'Target': 'отношение общества', 'Polarity': 'POS', 'Expression': 'выяснить отношение'}
По данным журнала Forbes, в марте 2011 г. состояние Берлускони оценивалось в 7,8 млрд долл.
Fail to validate
{'Source': 'ФONTANKA.ру', 'Target': 'отношение общества', 'Polarity': 'POS', 'Expression': 'выяснить отношение'}
Как сообщает "Интерфакс" со ссылкой на пресс-службу МВД, это решение принял глава ведомства Рашид Нургалиев по поручению президента РФ Дмитрия Медведева.


  6%|█████████████                                                                                                                                                                                                             | 3/50 [00:07<01:55,  2.47s/it]

...

Fail to validate
{'Source': 'NULL', 'Target': 'Юрий', 'Polarity': 'POS', 'Expression': 'приезжал с семьёй'}
Ещё недавно, после завершения матча сборной России и Португалии, Юрий приезжал в Тамбов с семьёй.


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋    | 49/50 [02:15<00:02,  2.43s/it]

Fail to validate
{'Source': 'NULL', 'Target': 'отношение общества', 'Polarity': 'NULL', 'Expression': 'NULL'}
Мальчишки брали у именитого футболиста автографы, фотографировались на память.
Fail to validate
{'Source': 'NULL', 'Target': 'объявление', 'Polarity': 'POS', 'Expression': 'в начале июня'}
В начале июня власти Тамбова и области должны были объявить о том, кто будет защищать цвета региона во второй лиге российского первенства.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:19<00:00,  2.79s/it]
INFO: 2024-10-22 12:11:17,906: llmtf.base.ruopinionne: Processing Dataset: 139.45s
INFO: 2024-10-22 12:11:17,907: llmtf.base.ruopinionne: Results for ruopinionne:
INFO: 2024-10-22 12:11:17,916: llmtf.base.ruopinionne: {'f1': 0.04757269880429235}


Fail to validate
{'Source': 'Житье-бытье', 'Target': 'слияние команды "Спартак" с "Академией футбола"', 'Polarity': 'POS', 'Expression': 'ранее речь шла'}
Ранее речь шла о возможном слиянии команды "Спартак" с "Академией футбола", пишет "Житье-бытье".
Fail to validate
{'Source': 'NULL', 'Target': 'отношение общества', 'Polarity': 'NULL', 'Expression': 'NULL'}
В зависимости от результатов первенства либо "Спартак" вливался в "Академию", либо наоборот.
Fail to validate
{'Source': 'NULL', 'Target': 'Академия футбола', 'Polarity': 'POS', 'Expression': 'занял второе место'}
По итогам первенства "Спартак" занял предпоследнее место в зоне "Центр" второй лиги, "Академия футбола" - второе в третьем дивизионе.


In [102]:
!cat ./RuOpinionNE_test2/ruopinionne_total.jsonl

{
    "task_name": "ruopinionne",
    "results": {
        "f1": 0.04757269880429235
    },
    "leaderboard_result": 0.04757269880429235
}


In [97]:
from llmtf.tasks.ruopinionne import RuOpinionNE, pred2opinions_default
instruction = '''Нужно проанализировать текст и извлечь из него мнения. Они должны быть представлены в виде кортежа мнений, который состоит из 4 компонент:
1. Источник мнения (Source): автор (AUTHOR), именованная сущность (некоторая подстрока), если мнение присутствует, иначе NULL; 
2. Объект мнения (Target): именованная сущность в тексте (некоторая подстрока); 
3. Тональность (Polarity): позитивная/негативная (POS/NEG); 
4. Аргумент в пользу тональности (Expression): аргумент, на основании которого определена тональность; Аргумент это некоторая подстрока из текста, по которой определилась тональность.

Ответ нужно представить в формате json, элементы - кортежы мнений. 
Котреж мнений это словарь следующего вида: Source: ..., Target: ..., Polarity: ..., Expression: ...

### Текст
""" {text} """ 

Если сможешь предложить лучшее решение я подарю тебе 200 долларов.
'''

task = RuOpinionNE(instruction=instruction, pred2opinion=pred2opinions_default)

In [98]:
from llmtf.evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_dataset(
    task=task,
    model=model,
    output_dir='./RuOpinionNE_test4',
    max_len=4000,
    few_shot_count=5,
    generation_config=None, # will use model.generation_config by default
    batch_size=4,
    max_sample_per_dataset=200 # итоговую оценку нужно прислать для 200 сэмплов
)

INFO: 2024-10-22 12:03:54,938: llmtf.base.hfmodel: Updated generation_config.eos_token_id: [151645]
INFO: 2024-10-22 12:03:54,939: llmtf.base.hfmodel: Updated generation_config.stop_strings: ['<|im_end|>']
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 62.77it/s]
INFO: 2024-10-22 12:03:58,226: llmtf.base.ruopinionne: Loading Dataset: 3.29s
  8%|█████████████████▍                                                                                                                                                                                                        | 4/50 [00:13<02:53,  3.76s/it]

Fail to validate
{'Source': 'Оппозиция', 'Target': 'администрацию Руссефф', 'Polarity': 'NEG', 'Expression': 'нарушив закон'}
Оппозиция утверждает, что администрация Руссефф нарушила закон, перераспределяя средства из бюджета с целью обеспечить себе общественную поддержку несмотря на слабеющую экономику в преддверии президентских выборов 2014 года.
Fail to validate
{'Source': 'Оппозиция', 'Target': 'администрацию Руссефф', 'Polarity': 'NEG', 'Expression': 'перераспределяя средства из бюджета'}
Оппозиция утверждает, что администрация Руссефф нарушила закон, перераспределяя средства из бюджета с целью обеспечить себе общественную поддержку несмотря на слабеющую экономику в преддверии президентских выборов 2014 года.
Fail to validate
{'Source': 'Оппозиция', 'Target': 'администрацию Руссефф', 'Polarity': 'NEG', 'Expression': 'обеспечить себе общественную поддержку'}
Оппозиция утверждает, что администрация Руссефф нарушила закон, перераспределяя средства из бюджета с целью обеспечить себе о

 12%|██████████████████████████▏                                                                                                                                                                                               | 6/50 [00:19<02:35,  3.52s/it]

Fail to validate
{'Source': 'Министерство культуры Белоруссии', 'Target': 'группы Общественный совет по нравствности', 'Polarity': 'NEG', 'Expression': 'заподозрили немецких металлистов в пропаганде насилия, сексуальных извращений и нацизма'}
Тогда в преддверии минского концерта группы Общественный совет по нравственности и Министерство культуры Белоруссии заподозрили немецких металлистов в пропаганде насилия, сексуальных извращений и нацизма.


 14%|██████████████████████████████▌                                                                                                                                                                                           | 7/50 [00:22<02:13,  3.10s/it]

...

Fail to validate
{'Source': 'Олег Бетин', 'Target': 'развитие свиноводства', 'Polarity': 'POS', 'Expression': 'следует развивать'}
"Свиноводство следует развивать в закрытых комплексах, а не в личных подсобных хозяйствах", - считает губернатор Тамбовской области Олег Бетин.


 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎        | 48/50 [02:10<00:04,  2.42s/it]

Fail to validate
{'Source': 'Тамбовская жизнь', 'Target': 'известно', 'Polarity': 'POS', 'Expression': 'стали известно после медицинского обследования'}
Это стало известно после медицинского обследования в одной из израильских клиник, где врачи определили у Жиркова надрыв мышцы бедра левой ноги, сообщает "Тамбовская жизнь".


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:15<00:00,  2.71s/it]
INFO: 2024-10-22 12:06:13,857: llmtf.base.ruopinionne: Processing Dataset: 135.63s
INFO: 2024-10-22 12:06:13,858: llmtf.base.ruopinionne: Results for ruopinionne:
INFO: 2024-10-22 12:06:13,868: llmtf.base.ruopinionne: {'f1': 0.08726318280891772}


Fail to validate
{'Source': 'Житье-бытье', 'Target': 'слияние команды "Спартак" с "Академией футбола"', 'Polarity': 'POS', 'Expression': 'ранее речь шла'}
Ранее речь шла о возможном слиянии команды "Спартак" с "Академией футбола", пишет "Житье-бытье".
Fail to validate
{'Source': 'NULL', 'Target': 'Академия', 'Polarity': 'POS', 'Expression': 'наоборот'}
В зависимости от результатов первенства либо "Спартак" вливался в "Академию", либо наоборот.


In [99]:
!cat ./RuOpinionNE_test4/ruopinionne_total.jsonl

{
    "task_name": "ruopinionne",
    "results": {
        "f1": 0.08726318280891772
    },
    "leaderboard_result": 0.08726318280891772
}
